In [1]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import time
import numpy as np

In [2]:
tornado_dispatch = pd.read_csv('../data/Computer Aided Dispatch Data_Tornado Incidents 030320 - Copy2.csv')

In [3]:
zipcodes = gpd.read_file('../data/zipcodes.geojson')

In [4]:
tornado_dispatch['LandUseType'].value_counts()

Residential                148
Other/Unknown               62
Commercial                  36
Public - Non-Commercial     14
Name: LandUseType, dtype: int64

In [5]:
conditions = [
    tornado_dispatch['LandUseType'] == 'Residential',
    tornado_dispatch['LandUseType'] == 'Commercial',
    tornado_dispatch['LandUseType'] == 'Public - Non-Commercial',
    tornado_dispatch['LandUseType'] == 'Other/Unknown'
    ]

values = ['#9F0606', '#D71FDE', '#DE771F', '#EAA5EC']

tornado_dispatch['color'] = np.select(conditions, values)

In [6]:
tornado_dispatch['color'].value_counts()

#9F0606    148
#EAA5EC     62
#D71FDE     36
#DE771F     14
Name: color, dtype: int64

In [7]:
tornado_dispatch['geometry'] = tornado_dispatch.apply(lambda x: Point((float(x.Longitude1), float(x.Latitude1))), axis=1)

In [8]:
tornado_geo = gpd.GeoDataFrame(tornado_dispatch, crs=zipcodes.crs, geometry = tornado_dispatch['geometry'])

In [9]:
zipcodes = zipcodes[['zip', 'po_name', 'geometry']]

In [10]:
tornado_geo = gpd.sjoin(tornado_geo, zipcodes, op = 'within')

In [11]:
polygon37213 = zipcodes.loc[zipcodes['zip'] == '37213']

In [12]:
polygon37213.geometry.centroid

<ipython-input-12-3a0394a5dad1>:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  polygon37213.geometry.centroid


31    POINT (-86.76851 36.16713)
dtype: geometry

In [13]:
center = polygon37213.geometry.centroid[31]
print(center)

POINT (-86.76850874477554 36.16713345170994)


<ipython-input-13-8954b78ae4d7>:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = polygon37213.geometry.centroid[31]


In [14]:
area_center = [center.y, center.x]

In [15]:
print(area_center)

[36.167133451709944, -86.76850874477554]


In [16]:
map_37213 = folium.Map(location = area_center, zoom_start = 12)
folium.TileLayer('stamentoner').add_to(map_37213)

In [17]:
for row_index, row_values in tornado_geo.iterrows():
    loc = [row_values['Latitude1'], row_values['Longitude1']]
    pop = str(row_values[['IncidentTypeDescription1']])
    marker = folium.Circle(
        location = loc,
        radius=100,
        popup = pop,
        fill=True,
        color=row_values['color']) 
    
    marker.add_to(map_37213)
    
map_37213.save('../maps/map_37213.html')
map_37213

In [18]:
folium.Map?

cluster_map_37213 = folium.Map(location = area_center, zoom_start = 12)

marker_cluster = MarkerCluster().add_to(cluster_map_37213)

for row_index, row_values in tornado_geo.iterrows():
    loc = [row_values['Latitude1'], row_values['Longitude1']]
    pop = str(row_values[['IncidentDate', 'IncidentTypeDescription1']])
    marker = folium.Marker(
        location = loc, 
        popup = pop) 
    
    marker.add_to(marker_cluster)
    
cluster_map_37213.save('../maps/cluster_37213.html')
cluster_map_37213
